In [1]:
import numpy as np
import pandas as pd

In [2]:
main_file_path = "../main_data/{}"

## Originale Schedule

In [4]:
file_path = main_file_path.format("0204b_tardiness_max_schedule_og.csv")
file_path

'../main_data/0204b_tardiness_max_schedule_og.csv'

In [5]:
df_plan = pd.read_csv(file_path)
df_plan = df_plan.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan

,Job,Operation,Arrival,Deadline,Machine,Start,Processing Time,End,Tardiness
0,Job_000,0,0.00,1246.644120,M0,0.00,29.0,29.00,0
1,Job_000,1,0.00,1246.644120,M1,29.00,78.0,107.00,0
2,Job_000,2,0.00,1246.644120,M2,107.00,9.0,116.00,0
3,Job_000,3,0.00,1246.644120,M3,116.00,36.0,152.00,0
4,Job_000,4,0.00,1246.644120,M4,152.00,49.0,201.00,0
...,...,...,...,...,...,...,...,...,...
375,Job_037,5,2869.61,4573.882975,M9,4247.28,76.0,4323.28,0
376,Job_037,6,2869.61,4573.882975,M5,4323.28,47.0,4370.28,0
377,Job_037,7,2869.61,4573.882975,M3,4370.28,52.0,4422.28,0
378,Job_037,8,2869.61,4573.882975,M4,4422.28,90.0,4512.28,0


## Revidierter Schedule (angepasster Zeitplan)
(Simulation und der neu geplante Rest)

In [7]:
file_path = main_file_path.format("0204b_tardiness_max_schedule_revised.csv")
file_path = main_file_path.format("0204b_tardiness_max_schedule_revised_with_sim.csv")
file_path

'../main_data/0204b_tardiness_max_schedule_revised_with_sim.csv'

In [8]:
df_revised = pd.read_csv(file_path)
df_revised = df_revised.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_revised 

,Job,Operation,Arrival,Machine,Start,Processing Time,Flow time,End,Deadline,Tardiness
0,Job_000,0,0.00,M0,0.00,13.20,13.20,13.20,NaN,NaN
1,Job_000,1,0.00,M1,29.00,96.46,125.46,125.46,NaN,NaN
2,Job_000,2,0.00,M2,125.46,9.26,134.72,134.72,NaN,NaN
3,Job_000,3,0.00,M3,134.72,30.43,165.15,165.15,NaN,NaN
4,Job_000,4,0.00,M4,165.15,34.96,200.11,200.11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
375,Job_037,5,2869.61,M9,4241.09,76.00,NaN,4317.09,4573.882975,0.0
376,Job_037,6,2869.61,M5,4317.09,47.00,NaN,4364.09,4573.882975,0.0
377,Job_037,7,2869.61,M3,4386.88,52.00,NaN,4438.88,4573.882975,0.0
378,Job_037,8,2869.61,M4,4438.88,90.00,NaN,4528.88,4573.882975,0.0


# Schedule Nervousness nach Hwangbo et al.

## Schedule-Nervousness-Index $SN$

$$
SN \;=\; P_A \;+\; P_R \;+\; P_T \;+\; P_U
$$

- **$P_A$** – *Addition-Term*: Vorgänge, die nach dem Rescheduling-Zeitpunkt $T_1$ neu eingeplant wurden.  
- **$P_R$** – *Removal-Term*: Vorgänge aus dem Basisplan, die nach $T_1$ gestrichen wurden.  
- **$P_T$** – *Time-Shift-Term*: Verschiebungen der Startzeit bereits geplanter Vorgänge.  
- **$P_U$** – *Unit-Shift-Term*: Verlagerungen eines Vorgangs auf eine andere Maschine  
  (im starren Flow-Shop ist $P_U = 0$).

> **Alle vier Terme werden durch eine fallende Gewichtungsfunktion $g(t)$ skaliert.**  
> Änderungen, die **kurz nach $T_1$** (Reschedule-Anfang) liegen, erhalten fast das volle Gewicht ($g(t)\!\approx\!1$)  
> und wirken sich somit stark auf $SN$ aus, während Änderungen **weit in der Zukunft** nur gering zählen.


In [11]:
# Vorbereitung: Operation je Job
#for df in (df_plan, df_revised):
#    if 'Operation' not in df.columns:  
#        df['Operation'] = df.groupby('Job').cumcount()

In [12]:
df_plan.head()

,Job,Operation,Arrival,Deadline,Machine,Start,Processing Time,End,Tardiness
0,Job_000,0,0.0,1246.64412,M0,0.0,29.0,29.0,0
1,Job_000,1,0.0,1246.64412,M1,29.0,78.0,107.0,0
2,Job_000,2,0.0,1246.64412,M2,107.0,9.0,116.0,0
3,Job_000,3,0.0,1246.64412,M3,116.0,36.0,152.0,0
4,Job_000,4,0.0,1246.64412,M4,152.0,49.0,201.0,0


In [13]:
df_revised.head()

,Job,Operation,Arrival,Machine,Start,Processing Time,Flow time,End,Deadline,Tardiness
0,Job_000,0,0.0,M0,0.00,13.20,13.20,13.20,NaN,NaN
1,Job_000,1,0.0,M1,29.00,96.46,125.46,125.46,NaN,NaN
2,Job_000,2,0.0,M2,125.46,9.26,134.72,134.72,NaN,NaN
3,Job_000,3,0.0,M3,134.72,30.43,165.15,165.15,NaN,NaN
4,Job_000,4,0.0,M4,165.15,34.96,200.11,200.11,NaN,NaN


In [14]:
# -----------------------------------------------------------
# Hilfsfunktion: logarithmische Gewichtung g(t)  (Formel 24)
# -----------------------------------------------------------
def g(t: float, T: float, T1: float) -> float:
    """
    Logarithmisch fallende Gewichtungsfunktion g(t)
    t   : Startzeit des Vorgangs im Basisplan   (t ≥ T1)
    T   : Gesamter Planhorizont  (z. B. max(df_plan['End']))
    T1  : Rescheduling-Zeitpunkt (Ende der 1-Tages-Simulation)
    """
    denom = np.log(T) - np.log(T1)
    return (np.log(T) - np.log(t)) / denom          # identisch zu Formel 24

## a) Time-Shift-Term $P_T$ (Formel 22)

$$
P_T
  \;=\;
  \sum_{i}
  \sum_{\,j,\,j' \in J_i}
  \sum_{\substack{t,\,t' \in T \setminus T_1 \\ t \neq t'}}
  g(t)\,\lvert t - t' \rvert \, U_{ijj'tt'}
$$



- $\displaystyle\sum_i$ läuft über **alle Jobs** $i$.  
- $\displaystyle\sum_{j,j'}$ läuft über das **Maschinen­paar** $(j,j')$ desselben Arbeitsgangs  
  (alte Maschine $j$, neue Maschine $j'$).  
- $\displaystyle\sum_{t,t'}$ läuft über die **Startzeiten** $t$ (Basisplan, $t\ge T_1$) und $t'$ (Re-Plan) desselben Vorgangs.  
- $g(t)$ ist die **logarithmische Gewichtung** (groß kurz nach $T_1$, klein in ferner Zukunft).  
- $\lvert t-t'\rvert$ ist der **absolute Startzeit-Versatz**.  
- $U_{ijj'tt'}$ ist ein **Indikator**: $1$, wenn der Vorgang von $(i,j,t)$ nach $(i,j',t')$ wandert, sonst $0$.


### Naive Umsetzung der Formel 22

In [19]:
def compute_P_T_naive(df_plan: pd.DataFrame,
                       df_revised: pd.DataFrame,
                       T1: float) -> float:
    """
    Berechnet den Time-Shift-Term P_T exakt nach Formel 22
    mit drei expliziten Schleifen:  Σ_i  Σ_{j,j'}  Σ_{t,t'}.

    Voraussetzungen
    ---------------
    • Beide DataFrames besitzen die Spalte 'Operation'
      (z. B. vorher mit df.groupby('Job').cumcount() erzeugt).
    • Zeiteinheiten in df_* und T1 identisch.
    """
    # Planungshorizont T (größtes End-Zeit-Stempel aus beiden Plänen)
    T = max(df_plan['End'].max(), df_revised['End'].max())

    P_T = 0.0                                             # Akkumulator

    # -------- Σ_i  (über Jobs) ----------------------------------------
    for job in sorted(set(df_plan['Job']).intersection(df_revised['Job'])):

        df_plan_job = df_plan[df_plan['Job'] == job]
        df_rev_job  = df_revised[df_revised['Job'] == job]

        # -------- Σ_{j}  (Maschinen im Basisplan) ---------------------
        for _, row_plan in df_plan_job.iterrows():

            t  = row_plan['Start']                        # Startzeit im Basisplan
            if t < T1:                                    # nur Vorgänge nach T1
                continue

            j  = row_plan['Machine']                      # Maschine im Basisplan
            op = row_plan['Operation']                           # Operations-Index

            # passendes Op im Re-Plan suchen (gleicher Job & Op)
            df_rev_op = df_rev_job[df_rev_job['Operation'] == op]
            if df_rev_op.empty:
                continue                                  # U = 0: nichts addieren

            # -------- Σ_{j'}  (Maschinen im Re-Plan) ------------------
            for _, row_rev in df_rev_op.iterrows():

                j_prime  = row_rev['Machine']             # Maschine im Re-Plan
                t_prime  = row_rev['Start']               # Startzeit im Re-Plan

                # -------- Σ_{t'}  (es gibt nur genau dieses t') -------
                term = g(t, T, T1) * abs(t - t_prime)     # g(t)·|t−t'|
                P_T += term                               # U_ijj'tt' = 1 genau hier

                # Hinweis: Es gibt zu jedem (i,op) genau eine Zeile in
                #          beiden Plänen; damit läuft die innere Schleife
                #          meist genau einmal. Sie ist nur da, um die
                #          Schreibweise Σ_{j'} Σ_{t'} explizit zu zeigen.

    return P_T

In [20]:
T1 = 24 * 60   # z. B. 1 Tag in Minuten
P_T = compute_P_T_naive(df_plan, df_revised, T1)
print(f"P_T (naiv) = {P_T:.2f}")

P_T (naiv) = 20634.67


### Vereinfachte Umsetzung (dank Dataframe-Struktur)

In [22]:
# -----------------------------------------------------------
# P_T berechnen  +  DataFrame zurückgeben
# -----------------------------------------------------------
def compute_P_T(df_plan: pd.DataFrame,
                df_revised: pd.DataFrame,
                T1: float,
                verbose: bool = True):
    """
    Liefert
    -------
    P_T     : float
        Time-Shift-Index nach Formel 22.
    details : pd.DataFrame
        Zeilenweiser Beitrag je Operation mit Spalten:
        Job, Op, Start_plan, Start_rev, delta_t, g, contrib
    """

    # 1) Zusammenführen & Filter
    details = (
        df_plan[['Job', 'Operation', 'Start']].rename(columns={'Start': 'Start_plan'})
        .merge(
            df_revised[['Job', 'Operation', 'Start']].rename(columns={'Start': 'Start_rev'}),
            on=['Job', 'Operation'], how='inner'
        )
        .query('Start_plan >= @T1')
        .assign(delta_t=lambda d: (d.Start_plan - d.Start_rev).abs())
    )

    # 2) g(t) und Beitrag
    T = max(df_plan['End'].max(), df_revised['End'].max())
    details['g']       = details['Start_plan'].apply(lambda t: g(t, T, T1))
    details['contrib'] = details['g'] * details['delta_t']
    P_T = details['contrib'].sum()

    # 3) Optionale Debug-Ausgabe (wie Version 2, leicht gekürzt)
    if verbose:
        print("=" * 70)
        print("Debug-Info  compute_P_T".center(70))
        print("=" * 70)
        print(f"{'Vorgänge nach T1':<30}: {len(details):>10}")
        print(f"{'Planungshorizont T':<30}: {T:>10.2f}")
        print("-" * 70)
        print(f"{'Metric':<25}{'Min':>12}{'Mean':>12}{'Max':>12}")
        print("-" * 70)
        print(f"{'delta_t (|t−t′|)':<25}{details['delta_t'].min():>12.2f}"
              f"{details['delta_t'].mean():>12.2f}{details['delta_t'].max():>12.2f}")
        print(f"{'g(t)':<25}{details['g'].min():>12.3f}"
              f"{details['g'].mean():>12.3f}{details['g'].max():>12.3f}")
        print("-" * 70)
        print("Beispiel-Zeilen (Top 5):")
        print(
            details[['Job', 'Operation', 'Start_plan', 'Start_rev',
                     'delta_t', 'g', 'contrib']]
            .head()
            .to_string(index=False, formatters={
                'Start_plan': '{:,.2f}'.format,
                'Start_rev' : '{:,.2f}'.format,
                'delta_t'   : '{:,.2f}'.format,
                'g'         : '{:,.3f}'.format,
                'contrib'   : '{:,.2f}'.format
            })
        )
        print("-" * 70)
        print(f"{'P_T (Summe)':<25}: {P_T:>12.2f}")
        print("=" * 70)

    return P_T, details

In [23]:
P_T, df_details = compute_P_T(df_plan, df_revised, T1, verbose=True)

                       Debug-Info  compute_P_T                        
Vorgänge nach T1              :        273
Planungshorizont T            :    4630.09
----------------------------------------------------------------------
Metric                            Min        Mean         Max
----------------------------------------------------------------------
delta_t (|t−t′|)                 0.00      137.38      594.19
g(t)                            0.016       0.466       1.000
----------------------------------------------------------------------
Beispiel-Zeilen (Top 5):
    Job  Operation Start_plan Start_rev delta_t     g contrib
Job_003          8   1,443.28  1,593.09  149.81 0.998  149.52
Job_003          9   1,466.28  1,615.09  148.81 0.985  146.51
Job_008          2   1,696.28  1,593.09  103.19 0.860   88.72
Job_008          3   1,793.04  1,721.09   71.95 0.812   58.44
Job_008          4   1,844.04  1,772.09   71.95 0.788   56.71
-----------------------------------------------

In [24]:
df_details

,Job,Operation,Start_plan,Start_rev,delta_t,g,contrib
38,Job_003,8,1443.28,1593.09,149.81,0.998052,149.518163
39,Job_003,9,1466.28,1615.09,148.81,0.984515,146.505674
82,Job_008,2,1696.28,1593.09,103.19,0.859757,88.718320
83,Job_008,3,1793.04,1721.09,71.95,0.812259,58.442008
84,Job_008,4,1844.04,1772.09,71.95,0.788245,56.714227
...,...,...,...,...,...,...,...
375,Job_037,5,4247.28,4241.09,6.19,0.073889,0.457374
376,Job_037,6,4323.28,4317.09,6.19,0.058704,0.363376
377,Job_037,7,4370.28,4386.88,16.60,0.049446,0.820798
378,Job_037,8,4422.28,4438.88,16.60,0.039318,0.652680


## b) Addition-Term $P_A$

$$
P_A
  = \sum_{i}
    \sum_{j\in J_i}
    \sum_{t\in T_2}
    g(t)\,Y_{ijt}
$$

- $\displaystyle\sum_i$ – über **alle Jobs** $i$.  
- $\displaystyle\sum_{j\in J_i}$ – über **alle Maschinen** $j$, die Job $i$ bearbeiten kann.  
- $\displaystyle\sum_{t\in T_2}$ – über **alle Zeitperioden** $t$ **nach** dem Rescheduling-Zeitpunkt ($T_2 = \{t\mid t \ge T_1\}$).  
- $g(t)$ – logarithmische Gewichtung (wie in Formel 24).  
- $Y_{ijt}$ – **Indikator**: $1$, wenn Vorgang $(i,j,t)$ **neu hinzugefügt** wurde (im Re-Plan vorhanden, im Basisplan nicht), sonst $0$.


In [27]:
def compute_P_A_naive(df_plan: pd.DataFrame,
                      df_revised: pd.DataFrame,
                      T1: float) -> float:
    """
    Berechnet P_A exakt nach Formel 20 mit drei verschachtelten Schleifen.

    Voraussetzungen
    ---------------
    • Beide DataFrames besitzen die Spalte 'Operation'
      (z. B. vorher via df.groupby('Job').cumcount() erzeugt).
    • Zeiteinheit in 'Start', 'End' und T1 ist identisch.
    """

    # Planungshorizont |T| : größtes End-Zeit-Stempel aus beiden Plänen
    T = max(df_plan['End'].max(), df_revised['End'].max())

    # ----------  Hilfs­struktur ------------------------------
    # Menge aller Basis-Operationen, die NACH T1 starten
    basis_ops_T2 = set(
        zip(df_plan.loc[df_plan['Start'] >= T1, 'Job'],
            df_plan.loc[df_plan['Start'] >= T1, 'Operation'])
    )

    P_A = 0.0                                    # Akkumulator

    # ----------  Σ_i  (über Jobs) ----------------------------
    for job in sorted(df_revised['Job'].unique()):

        # alle Reihen dieses Jobs im revidierten Plan
        df_rev_job = df_revised[df_revised['Job'] == job]

        # ----------  Σ_j  (über Maschinen) -------------------
        for machine in df_rev_job['Machine'].unique():

            df_rev_job_m = df_rev_job[df_rev_job['Machine'] == machine]

            # ----------  Σ_t  (über Startzeiten t ≥ T1) ------
            for _, row in df_rev_job_m.iterrows():

                t   = row['Start']
                op  = row['Operation']

                if t < T1:                       # nur Zeitpunkte in T2 betrachten
                    continue

                # Y_{ijt} = 1, falls Arbeitsgang im Basisplan NICHT in T2 existiert
                if (job, op) not in basis_ops_T2:
                    P_A += g(t, T, T1)           # g(t) * Y  (Y = 1)

                # Andernfalls Y = 0  →  nichts addieren

    return P_A

In [28]:
T1 = 24 * 60   # z. B. 1 Tag in Minuten

P_A = compute_P_A_naive(df_plan, df_revised, T1)
print(f"P_A = {P_A:.2f}")

P_A = 11.18


In [29]:
def compute_P_A(df_plan: pd.DataFrame,
                df_revised: pd.DataFrame,
                T1: float,
                verbose: bool = True):
    """
    Liefert
    -------
    P_A     : float
        Addition-Index nach Formel 20.
    details : pd.DataFrame
        Alle Vorgänge mit Y_{ijt}=1 und ihrem g(t)-Beitrag.
                Spalten: Job, Op, Start, g
    Voraussetzungen
    ---------------
    • Beide DataFrames besitzen die Spalte 'Operation'
      (z. B.: df['Operation'] = df.groupby('Job').cumcount()).
    • 'Start', 'End' und T1 sind in derselben Zeiteinheit.
    """

    # 1) Teilmengen für T₂ (Start >= T1)
    plan_T2    = df_plan   .loc[df_plan   ['Start'] >= T1, ['Job', 'Operation']]
    revised_T2 = df_revised.loc[df_revised['Start'] >= T1, ['Job', 'Operation', 'Start']]

    # 2) Anti-Join: Re-Plan-Ops, die im Basis-T₂ nicht existieren → Y=1
    additions = revised_T2.merge(plan_T2,
                                 on=['Job', 'Operation'],
                                 how='left',
                                 indicator=True)
    additions = additions[additions['_merge'] == 'left_only'] \
                .drop(columns='_merge') \
                .reset_index(drop=True)

    # 3) g(t) und P_A
    T = max(df_plan['End'].max(), df_revised['End'].max())
    additions['g'] = additions['Start'].apply(lambda t: g(t, T, T1))
    P_A = additions['g'].sum()

    # 4) Debug-Ausgabe
    if verbose:
        print("=" * 60)
        print("Debug-Info  compute_P_A".center(60))
        print("=" * 60)
        print(f"Neu hinzugefügte Vorgänge (Y=1): {len(additions):>6}")
        if not additions.empty:
            print(f"g(t) – Min/Ø/Max : "
                  f"{additions['g'].min():.3f} / "
                  f"{additions['g'].mean():.3f} / "
                  f"{additions['g'].max():.3f}")
        print(f"P_A (Summe g)      : {P_A:.2f}")
        print("=" * 60)

    return P_A, additions


In [30]:
import numpy as np
import pandas as pd

# -----------------------------------------------------------'Op'
# g(t) – logarithmische Gewichtung (Formel 24)
# -----------------------------------------------------------
def g(t: float, T: float, T1: float) -> float:
    return (np.log(T) - np.log(t)) / (np.log(T) - np.log(T1))


# -----------------------------------------------------------
# Addition-Term P_A  (Formel 20) – vektorisiert + sauberes Print-Layout
# -----------------------------------------------------------
def compute_P_A(df_plan: pd.DataFrame,
                df_revised: pd.DataFrame,
                T1: float,
                verbose: bool = True):
    """
    Rückgabe
    --------
    P_A     : float
    details : pd.DataFrame  (Job, Op, Start, g)  – alle Y_{ijt}=1-Fälle
    """
    # 1) Teilmengen (T₂-Bereich)
    plan_T2    = df_plan   .loc[df_plan   ['Start'] >= T1, ['Job', 'Operation']]
    revised_T2 = df_revised.loc[df_revised['Start'] >= T1, ['Job', 'Operation', 'Start']]

    # 2) Anti-Join → nur Vorgänge, die neu nach T1 erscheinen
    additions = (
        revised_T2
        .merge(plan_T2, on=['Job', 'Operation'], how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns='_merge')
        .reset_index(drop=True)
    )

    # 3) g(t) und P_A
    T = max(df_plan['End'].max(), df_revised['End'].max())
    additions['g'] = additions['Start'].apply(lambda t: g(t, T, T1))
    P_A = additions['g'].sum()

    # 4) Schönes Debug-Layout
    if verbose:
        print("=" * 70)
        print("Debug-Info  compute_P_A".center(70))
        print("=" * 70)
        print(f"{'Neu hinzugefügte Vorgänge (Y=1)':<40}: {len(additions):>10}")
        print(f"{'Planungshorizont T':<40}: {T:>10.2f}")
        print("-" * 70)
        if not additions.empty:
            print(f"{'g(t)   (Min / Ø / Max)':<40}"
                  f"{additions['g'].min():>8.3f}"
                  f"{additions['g'].mean():>8.3f}"
                  f"{additions['g'].max():>8.3f}")
            print("-" * 70)
            print("Beispiel-Zeilen (Top 5):")
            print(
                additions[['Job', 'Operation', 'Start', 'g']]
                .head()
                .to_string(index=False,
                           formatters={'Start': '{:,.2f}'.format,
                                       'g'    : '{:,.3f}'.format})
            )
        else:
            print("Keine Additionen nach T1 gefunden.")
        print("-" * 70)
        print(f"{'P_A (Summe g)':<40}: {P_A:>10.2f}")
        print("=" * 70)

    return P_A, additions.sort_values(by=["Job", "Operation"])


In [31]:
T1 = 24 * 60            # z. B. 1 Tag in Minuten

P_A, df_additions = compute_P_A(df_plan, df_revised, T1, verbose=True)

                       Debug-Info  compute_P_A                        
Neu hinzugefügte Vorgänge (Y=1)         :         12
Planungshorizont T                      :    4630.09
----------------------------------------------------------------------
g(t)   (Min / Ø / Max)                     0.822   0.931   0.968
----------------------------------------------------------------------
Beispiel-Zeilen (Top 5):
    Job  Operation    Start     g
Job_003          7 1,495.09 0.968
Job_007          6 1,498.75 0.966
Job_007          7 1,517.75 0.955
Job_007          8 1,565.75 0.928
Job_007          9 1,669.09 0.874
----------------------------------------------------------------------
P_A (Summe g)                           :      11.18


In [32]:
df_additions

,Job,Operation,Start,g
0,Job_003,7,1495.09,0.967855
1,Job_007,6,1498.75,0.965761
2,Job_007,7,1517.75,0.954975
3,Job_007,8,1565.75,0.928316
4,Job_007,9,1669.09,0.873593
5,Job_009,3,1498.75,0.965761
6,Job_009,4,1517.75,0.954975
7,Job_009,5,1687.09,0.864408
8,Job_009,6,1772.09,0.822322
9,Job_010,5,1505.75,0.961772


## c) Removal-Term $P_R$

$$
P_R
  = \sum_{i}
    \sum_{j\in J_i}
    \sum_{\substack{t\in T_2\\ \Omega X_{ijt}=1}}
    g(t)\,\bigl(1 - Z_{ijt}\bigr)
$$

- $\displaystyle\sum_i$ läuft über **alle Jobs** $i$.  
- $\displaystyle\sum_{j\in J_i}$ läuft über alle **Maschinen** $j$, auf denen Job $i$ bearbeitet werden kann.  
- $\displaystyle\sum_{t\in T_2,\;\Omega X_{ijt}=1}$ läuft über alle **Startzeiten** $t\ge T_1$, bei denen der Vorgang $(i,j,t)$ im **Basisplan** existiert ($\Omega X_{ijt}=1$).  
- $g(t)$ ist die **logarithmische Gewichtung** (Formel 24): groß kurz nach $T_1$, klein in ferner Zukunft.  
- $Z_{ijt}$ ist der Indikator “Vorgang $(i,j,t)$ ist auch im **revidierten** Plan vorhanden”.  
  Damit gilt $1 - Z_{ijt}=1$, wenn der Vorgang **gestrichen** wurde, sonst 0.

In [35]:
# -----------------------------------------------------------
# Naive Referenzfunktion  P_R  (Formel 21)
# -----------------------------------------------------------
def compute_P_R_naive(df_plan: pd.DataFrame,
                      df_revised: pd.DataFrame,
                      T1: float) -> float:
    """
    Berechnet den Removal-Index P_R genau nach den drei Summen
    Σ_i Σ_j Σ_t   aus Formel 21.

    Voraussetzungen
    ---------------
    • Beide DataFrames besitzen die Spalte 'Operation'
      (etwa via  df.groupby('Job').cumcount()).
    • 'Start', 'End' und T1 stehen in derselben Zeiteinheit.
    """

    # Planungshorizont |T|  (größtes End-Zeit-Stempel)
    T = max(df_plan['End'].max(), df_revised['End'].max())

    # Hilfs-Menge aller Vorgänge, die im RE-Plan (gleiches Job+Op) starten
    # – unabhängig von Maschine/Zeit.  Damit gilt Z_ijt = 1,
    #   falls überhaupt noch ein Pendant existiert.
    revised_ops = set(zip(df_revised['Job'], df_revised['Operation']))

    P_R = 0.0  # Akkumulator

    # -------------  Σ_i  -----------------------------------
    for job in sorted(df_plan['Job'].unique()):

        df_job_plan = df_plan[df_plan['Job'] == job]

        # -------------  Σ_j  --------------------------------
        for _, row in df_job_plan.iterrows():

            j   = row['Machine']
            t   = row['Start']
            op  = row['Operation']

            # nur Starts NACH T1 zählen (T₂-Bereich)
            if t < T1:
                continue

            # Z_{ijt} = 1, wenn der Arbeitsgang (Job,Op) im Re-Plan noch existiert
            Z_ijt = (job, op) in revised_ops

            # (1 - Z)  –   nur wenn Vorgang gestrichen
            if not Z_ijt:
                P_R += g(t, T, T1)          # g(t) · (1 - Z)   (hier = g(t))

            # existiert er noch, wird nichts addiert

    return P_R

In [36]:
P_R = compute_P_R_naive(df_plan, df_revised, T1)
print(f"P_R = {P_R:.2f}")

P_R = 0.00


In [37]:
# -----------------------------------------------------------
# Removal-Term P_R  –  vektorisiert und sauber formatiert
# -----------------------------------------------------------
def compute_P_R(df_plan: pd.DataFrame,
                df_revised: pd.DataFrame,
                T1: float,
                verbose: bool = True):
    """
    Berechnet den Removal-Index P_R (Formel 21) mit Pandas-Operationen.

    Rückgabe
    --------
    P_R      : float
        Summe g(t) aller gestrichenen Vorgänge.
    details  : pd.DataFrame
        Zeilen zu jedem gestrichenen Arbeitsgang  (Job, Op, Start_plan, g).

    Voraussetzungen
    ---------------
    • Beide DataFrames besitzen die Spalte 'Operation'
      (einmalig via  df['Operation'] = df.groupby('Job').cumcount()).
    • 'Start', 'End' und T1 teilen dieselbe Zeiteinheit.
    """

    # 1) Betrachtungsbereich T₂: Vorgänge, die im Basisplan NACH T1 starten
    basis_T2 = df_plan.loc[df_plan['Start'] >= T1,
                           ['Job', 'Operation', 'Start']].rename(columns={'Start': 'Start_plan'})

    # 2) Schlüssel aller Arbeitsgänge, die im Re-Plan IRGENDWO existieren  →  Z = 1
    revised_keys = df_revised[['Job', 'Operation']].drop_duplicates()

    # 3) Anti-Join → Basis-Vorgänge, die im Re-Plan fehlen  (1 − Z = 1  ⇒  Removal)
    removals = (
        basis_T2
        .merge(revised_keys, on=['Job', 'Operation'], how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns='_merge')
        .reset_index(drop=True)
    )

    # 4) g(t) und P_R
    T = max(df_plan['End'].max(), df_revised['End'].max())
    removals['g'] = removals['Start_plan'].apply(lambda t: g(t, T, T1))
    P_R = removals['g'].sum()

    # 5) Optional: schön formatierte Debug-Ausgabe
    if verbose:
        print("=" * 70)
        print("Debug-Info  compute_P_R".center(70))
        print("=" * 70)
        print(f"{'Gestrichene Vorgänge (Y=1)':<40}: {len(removals):>10}")
        print(f"{'Planungshorizont T':<40}: {T:>10.2f}")
        print("-" * 70)
        if not removals.empty:
            print(f"{'g(t)  (Min / Ø / Max)':<40}"
                  f"{removals['g'].min():>8.3f}"
                  f"{removals['g'].mean():>8.3f}"
                  f"{removals['g'].max():>8.3f}")
            print("-" * 70)
            print("Beispiel-Zeilen (Top 5):")
            print(
                removals[['Job', 'Operation', 'Start_plan', 'g']]
                .head()
                .to_string(
                    index=False,
                    formatters={'Start_plan': '{:,.2f}'.format,
                                'g'         : '{:,.3f}'.format}
                )
            )
        else:
            print("Keine Removals nach T1 gefunden.")
        print("-" * 70)
        print(f"{'P_R (Summe g)':<40}: {P_R:>10.2f}")
        print("=" * 70)

    return P_R, removals

In [38]:
P_R, df_removals = compute_P_R(df_plan, df_revised, T1, verbose=True)

                       Debug-Info  compute_P_R                        
Gestrichene Vorgänge (Y=1)              :          0
Planungshorizont T                      :    4630.09
----------------------------------------------------------------------
Keine Removals nach T1 gefunden.
----------------------------------------------------------------------
P_R (Summe g)                           :       0.00


## d) Reassignment-Term $P_U$

In [40]:
# da Flow Shop starr (jede Operation genau eine fest zugeordnete Maschine)
P_U = 0 

## Ergebnis

In [42]:
SN = P_A + P_R + P_T          # Flow-Shop → P_U entfällt
print(f"Schedule Nervousness SN = {SN:.2f}")

Schedule Nervousness SN = 20645.85


##### Beim Vergleich mehrer SN: 
Once the nervousness for the new schedule alternatives had been
 calculated, the calculated values for each element were standardized
 into a common distribution, effectively mitigating the influence of their
 respective magnitudes.